<a href="https://colab.research.google.com/github/mTwR0/Transcriber/blob/main/Audio_transcriber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

first, take audio, check for holds or silence for more than x seconds.

cut that into other audio segments or multiple files

In [ ]:
!pip install transformers datasets librosa pydub

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Path to your audio file
call_file = r'/content/drive/MyDrive/Colab Notebooks/test_call.wav'

# Verify the file exists
print("File exists:", os.path.exists(call_file))


In [ ]:
from pydub import AudioSegment, silence

# Load the audio file
audio = AudioSegment.from_file(call_file)

# Detect silent segments
silence_threshold = -50  # Silence threshold in dBFS
min_silence_length = 5000  # Minimum silence length in ms
silent_ranges = silence.detect_silence(audio, min_silence_len=min_silence_length, silence_thresh=silence_threshold)

# Convert silent ranges to seconds for readability
silent_ranges = [(start / 1000, end / 1000) for start, end in silent_ranges]
print("Silent ranges (start, end in seconds):", silent_ranges)

# Remove silence and combine non-silent parts
non_silent_audio = silence.split_on_silence(
    audio,
    min_silence_len=min_silence_length,
    silence_thresh=silence_threshold,
    keep_silence=200  # Keep small silence padding
)

# Combine the non-silent chunks into one audio segment
processed_audio = sum(non_silent_audio)

# Save the processed audio to a new file
processed_audio_path = r'/content/drive/MyDrive/Colab Notebooks/processed_audio.wav'
processed_audio.export(processed_audio_path, format="wav")
print(f"Processed audio saved to {processed_audio_path}")


In [ ]:
# List files in My Drive to verify your file is there
drive.mount('/content/drive')


print(os.path.exists(processed_audio_path))


In [ ]:
from transformers import pipeline

transcriber = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v3-turbo",
    return_timestamps=True
)

# Transcribe the processed audio
result = transcriber(processed_audio_path)

# Print the transcription
print("Transcription:", result["text"])
